<a href="https://colab.research.google.com/github/asribavan/cind820---capstone-project/blob/main/Initial%20Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
%matplotlib inline
inline_rc = dict(mpl.rcParams)

In [98]:
import os
print(os.listdir("/content/sample_data"))

['README.md', 'yelp_academic_dataset_business.json', '.ipynb_checkpoints']


In [99]:
users = []
with open('/content/sample_data/yelp_academic_dataset_business.json') as fl:
    for i, line in enumerate(fl):
        users.append(json.loads(line))
        if i+1 >= 100000:
            break
yelp_business = pd.DataFrame(users)
yelp_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [147]:
yelp_business = yelp_business.rename(columns = {'name': 'BusinessName'})
yelp_business.head()

,business_id,BusinessName,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [100]:
df_clean = yelp_business
print(yelp_business.shape)
print(yelp_business.dtypes)

(100000, 14)
business_id      object
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
stars           float64
review_count      int64
is_open           int64
attributes       object
categories       object
hours            object
dtype: object


In [101]:
import sklearn.feature_extraction.text as sk_text
Tfidf_vectorizer = sk_text.TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1), min_df = .05 , max_df = .95)

In [102]:
matrix = Tfidf_vectorizer.fit_transform(df_clean['city'])
tfidf_data = matrix.toarray()

In [103]:
tfidf_data

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [104]:
print(Tfidf_vectorizer.get_feature_names_out())

['indianapolis' 'philadelphia' 'tampa' 'tucson']


In [107]:
yelp_class = yelp_business[(yelp_business["stars"]==1) | (yelp_business["stars"]==5)]

In [108]:
reviews_yelp = yelp_business[yelp_business['stars'] != 3]

In [109]:
reviews_yelp['class'] = reviews_yelp['stars'].apply(lambda x : (1 if x>=3.5 else 0))
reviews_yelp.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,class
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,1
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",1
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",1
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",1
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",0


In [131]:
X=reviews_yelp["city"]
y=reviews_yelp["stars"]

In [115]:
from sklearn.feature_extraction.text import CountVectorizer

In [116]:
X=CountVectorizer().fit_transform(X)

In [185]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [118]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

In [119]:
mnb.fit(x_train,y_train)

MultinomialNB()

In [121]:
predict=mnb.predict(x_test)
from sklearn.metrics import confusion_matrix,classification_report

In [122]:
print(confusion_matrix(y_test, predict))
print("\n")
print(classification_report(y_test, predict))

[[   5  368]
 [  43 3227]]


              precision    recall  f1-score   support

         1.0       0.10      0.01      0.02       373
         5.0       0.90      0.99      0.94      3270

    accuracy                           0.89      3643
   macro avg       0.50      0.50      0.48      3643
weighted avg       0.82      0.89      0.85      3643



In [180]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(8498, 599)
(8498,)
(3643, 599)
(3643,)


Review Dataset

In [ ]:
users = []
with open('/content/sample_data/yelp_academic_dataset_review.json') as fl:
    for i, line in enumerate(fl):
        users.append(json.loads(line))
        if i+1 >= 100000:
            break
yelp_review = pd.DataFrame(users)
yelp_review.head()

In [ ]:
yelp_review.head()

In [ ]:
sns.boxplot(x = 'stars', y = 'polarity', data = yelp_review)

In [ ]:
print("Average polarity of 5-star review: ", yelp_business[yelp_business['stars'] == 5]['polarity'].mean())
print("Average polarity of 4-star review: ", yelp_business[yelp_business['stars'] == 4]['polarity'].mean())
print("Average polarity of 3-star review: ", yelp_business[yelp_business['stars'] == 3]['polarity'].mean())
print("Average polarity of 2-star review: ", yelp_business[yelp_business['stars'] == 2]['polarity'].mean())
print("Average polarity of 1-star review: ", yelp_business[yelp_business['stars'] == 1]['polarity'].mean())

In [ ]:
reviews = yelp_review[yelp_review['stars'] != 3]

In [ ]:
reviews['class'] = reviews['stars'].apply(lambda x : (1 if x>=4 else 0))
reviews.head()

In [ ]:
yelp_review.shape
yelp_review.stars.value_counts()

In [ ]:
yelp_dataset = yelp_business.append(yelp_review)
display(yelp_dataset)

In [ ]:
yelp_data = yelp_dataset
print(yelp_data.shape)
print(yelp_data.dtypes)
yelp_dataset.head

In [ ]:
import sklearn.feature_extraction.text as sk_text
tfidf_vector = sk_text.TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1), min_df = .05 , max_df = .85)

In [ ]:
matrix = tfidf_vector.fit_transform(yelp_data['all_review'])

In [ ]:
tfidf_data = matrix.toarray()
tfidf_data

SVM

In [191]:
X_train, X_test, y_train, y_test = train_test_split(yelp_data, yelp_data.stars, test_size=0.3,random_state=109)

In [ ]:
from sklearn import svm

clf = svm.SVC(kernel='linear') 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred, average='macro')

In [ ]:
plt.plot(y_pred_svm[10:20],label = 'Predicted')
plt.plot(y_test[10:20].values, label = 'Actual')

plt.legend()
plt.show()

KNN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(yelp_data, yelp_data.stars, test_size=0.3,random_state=109)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [ ]:
k_range = range(1, 11)
scores = {}
scores_list = []

for k in k_range:
  knn = KNeighborsClassifier(n_neighbors = k)
  knn.fit(X_train, y_train)
  y_pred = knn.predict(X_test)
  scores[k] = metrics.accuracy_score(y_test, y_pred)
  scores_list.append(metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_test,y_pred_knn,average='micro')

In [ ]:
plt.plot(k_range, scores_list)
plt.xlabel("Value of K")
plt.ylabel("Accuracy")

NLP

In [ ]:
yelp_class = yelp_data[(yelp_data["stars"]==1) | (yelp_data["stars"]==5)]

In [ ]:
texts=yelp_class["text"]
stars=yelp_class["stars"]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
X=CountVectorizer().fit_transform(texts)

Logistic Regression

In [ ]:
f1_score(y_test,y_pred_log_reg,average='micro')

In [ ]:
from sklearn.metrics import accuracy_score
print("accuracy: %.2f"
      % accuracy_score(y_test, y_pred_log_reg))

In [ ]:
plt.plot(y_pred_log_reg[:10],label = 'Predicted')
plt.plot(y_test[:10].values, label = 'Actual')

plt.legend()
plt.show()